In [1]:
## sentiment analysis 
## note that active directory is already set in thfunction_file_news_corpus
## Keep all files in the same active dir
## refer to line 48 and line 49 of the function file
## $ conda install -c https://conda.anaconda.org/sloria textblob
## refer to http://textblob.readthedocs.io/en/dev/install.html documentation 


from function_file_news_corpus import *
from textblob import TextBlob
def map_sentiment (text):
    analyzer = TextBlob(text)
    return analyzer.sentiment.polarity

In [2]:
news_df_topics = pd.read_pickle('news_df_topics_for_sentiment_analysis_final.pkl')
news_df_topics['sentiment'] = news_df_topics['text'].map(map_sentiment)

## convert data time to an amenable format, year_quarter  

def yr (text):
    return int(str(text)[0:4])

def mnth (text):
    return int(str(text)[4:6])

def qtr (text):
    text =  int(str(text)[4:6])
    if text <=3:
        return 1
    elif text <=6:
        return 2
    elif text <=9:
        return 3
    elif text <=9:
        return 3
    elif text <=12:
        return 4
    
    
def day (text):
    return int(str(text)[-2:])


news_df_topics['year'] = news_df_topics['date'].map(yr)
news_df_topics['month'] = news_df_topics['date'].map(mnth)
news_df_topics['quarter'] = news_df_topics['date'].map(qtr)
news_df_topics['day'] = news_df_topics['date'].map(day)
news_df_topics['year_quarter'] = news_df_topics['year'] + news_df_topics['quarter']/10


news_df_sentiment_topic = news_df_topics[['topic','sentiment','year_quarter','year','month','day']].copy(deep=True)

news_df_sentiment_topic['sentiment_type'] = np.where(news_df_sentiment_topic.sentiment>0, 1, -1)
news_df_sentiment_topic['sentiment_type'] = np.where(news_df_sentiment_topic.sentiment == 0, 
                                                     0,news_df_sentiment_topic.sentiment_type )
topic_key_final = pd.read_excel('topic_key_final.xlsx')
news_df_sentiment_topic = pd.merge(news_df_sentiment_topic,topic_key_final,on=['topic'])
news_df_sentiment_topic.head()

,topic,sentiment,year_quarter,year,month,day,sentiment_type,inf_topic,sub_topic
0,25,0.043311,2018.1,2018,3,24,1,general,rural_tribal_society
1,25,0.000000,2018.1,2018,3,1,0,general,rural_tribal_society
2,25,0.149471,2018.1,2018,1,12,1,general,rural_tribal_society
3,25,0.147527,2018.1,2018,1,11,1,general,rural_tribal_society
4,25,0.134573,2017.4,2017,12,30,1,general,rural_tribal_society


In [70]:
writer = pd.ExcelWriter('time_scatter_sentiment_topic.xlsx')
news_df_sentiment_topic.to_excel(writer, sheet_name='time_scatter_sentiment')
writer.save()


In [92]:
news_df_sentiment_topic['mean_sentiment_topic_quarter'] = news_df_sentiment_topic.groupby(['year_quarter', 'topic'])['sentiment'].transform('mean')

plotly_mean_sentiment = news_df_sentiment_topic[['topic', 'year_quarter',
                                                       'mean_sentiment_topic_quarter', 'inf_topic', 'sub_topic' ]].copy(deep=True)
         
plotly_mean_sentiment.drop_duplicates(inplace=True)

plotly_mean_sentiment = plotly_mean_sentiment.sort_values(['topic', 'year_quarter'], ascending=[1, 1])
plotly_mean_sentiment.head(n=20)

,topic,year_quarter,mean_sentiment_topic_quarter,inf_topic,sub_topic
919,0,2010.4,0.244444,aadhaar_related_crime,fake_card | misuse
918,0,2011.1,-0.106250,aadhaar_related_crime,fake_card | misuse
916,0,2011.3,0.118750,aadhaar_related_crime,fake_card | misuse
915,0,2012.1,0.027473,aadhaar_related_crime,fake_card | misuse
913,0,2012.2,0.005000,aadhaar_related_crime,fake_card | misuse
912,0,2012.3,-0.046429,aadhaar_related_crime,fake_card | misuse
911,0,2012.4,-0.350000,aadhaar_related_crime,fake_card | misuse
908,0,2013.3,-0.089167,aadhaar_related_crime,fake_card | misuse
907,0,2014.1,-0.110000,aadhaar_related_crime,fake_card | misuse
903,0,2014.2,-0.098830,aadhaar_related_crime,fake_card | misuse


In [93]:

merge_year_quarter = pd.read_excel('merge_year_quarter.xlsx')

df_1 = list(plotly_mean_sentiment['topic'].unique())
df_2 =list(merge_year_quarter['year_quarter'].unique())


dfn = pd.DataFrame([(x, y) for x in df_1 for y in df_2])
dfn.rename(columns={1:'year_quarter',0 :'topic' }, inplace=True)
plotly_mean_sentiment= pd.merge(plotly_mean_sentiment,dfn,how='right',on=['year_quarter','topic'])

#plotly_mean_sentiment.fillna(0, inplace = True)
writer = pd.ExcelWriter('plotly_mean_sentiment.xlsx')
plotly_mean_sentiment.to_excel(writer, sheet_name='plotly_mean_sentiment')
writer.save()

plotly_mean_sentiment.head()

,topic,year_quarter,mean_sentiment_topic_quarter,inf_topic,sub_topic
0,0,2010.4,0.244444,aadhaar_related_crime,fake_card | misuse
1,0,2011.1,-0.106250,aadhaar_related_crime,fake_card | misuse
2,0,2011.3,0.118750,aadhaar_related_crime,fake_card | misuse
3,0,2012.1,0.027473,aadhaar_related_crime,fake_card | misuse
4,0,2012.2,0.005000,aadhaar_related_crime,fake_card | misuse


In [94]:
## analysis for values grouped by quarter:

news_df_sentiment_topic['mean_sentiment_topic_quarter'] = news_df_sentiment_topic.groupby(['year_quarter', 'topic'])['sentiment_type'].transform('mean')

plotly_mean_sentiment = news_df_sentiment_topic[['topic', 'year_quarter',
                                                       'mean_sentiment_topic_quarter', 'inf_topic', 'sub_topic' ]].copy(deep=True)
         
plotly_mean_sentiment.drop_duplicates(inplace=True)

plotly_mean_sentiment = plotly_mean_sentiment.sort_values(['topic', 'year_quarter'], ascending=[1, 1])
plotly_mean_sentiment.head(n=20)


merge_year_quarter = pd.read_excel('merge_year_quarter.xlsx')

df_1 = list(plotly_mean_sentiment['topic'].unique())
df_2 =list(merge_year_quarter['year_quarter'].unique())


dfn = pd.DataFrame([(x, y) for x in df_1 for y in df_2])
dfn.rename(columns={1:'year_quarter',0 :'topic' }, inplace=True)
plotly_mean_sentiment= pd.merge(plotly_mean_sentiment,dfn,how='right',on=['year_quarter','topic'])

active_dir = "/Users/anirudhsyal/Desktop/Hindu_news/mean_positive_negative"
os.chdir (active_dir )


#plotly_mean_sentiment.fillna(0, inplace = True)
writer = pd.ExcelWriter('plotly_mean_sentiment.xlsx')
plotly_mean_sentiment.to_excel(writer, sheet_name='plotly_mean_sentiment')
writer.save()

plotly_mean_sentiment.head()


,topic,year_quarter,mean_sentiment_topic_quarter,inf_topic,sub_topic
0,0,2010.4,1.0,aadhaar_related_crime,fake_card | misuse
1,0,2011.1,-1.0,aadhaar_related_crime,fake_card | misuse
2,0,2011.3,1.0,aadhaar_related_crime,fake_card | misuse
3,0,2012.1,1.0,aadhaar_related_crime,fake_card | misuse
4,0,2012.2,1.0,aadhaar_related_crime,fake_card | misuse


In [33]:
## analysis for all values 

news_df = pd.read_pickle('final_clean_hindu_corpus.pkl')
news_df.drop(['DateTime'], axis = 1, inplace = True)

news_df['tokens'] = news_df['text'].map(tokenizer_tf_idf)
news_df['count_tokens'] = news_df['tokens'].map(len)
news_df_count_all = news_df[['aadhaar_count','num_date','count_tokens']].copy(deep=True)

writer = pd.ExcelWriter('news_df_count_all.xlsx')
news_df_count_all.to_excel(writer, sheet_name='news_df_count_all')
writer.save()

news_df.head()


,text,Heading,original_text,doct_no,aadhaar_count,num_date,tokens,count_tokens
0,passion commitment research bedrock effort aim...,from aspiration to action,passion commitment and research are the bedro...,1,3,20180324,"[passion, commitment, research, bedrock, effor...",393
1,sudden spurt abroad seek police clearance cert...,single window for police clearance certificates,after sudden spurt in people going abroad see...,2,1,20180320,"[sudden, spurt, abroad, seek, police, clearanc...",104
2,municipal commissioner hari narayanan instruct...,firm action to ensure waste segregation,municipal commissioner m. hari narayanan has ...,3,2,20180315,"[municipal, commissioner, hari, narayanan, ins...",85
3,despite recent reiteration unique_identificati...,despite clause services continue to be denied...,despite recent reiteration by the unique iden...,4,16,20180311,"[despite, recent, reiteration, unique_identifi...",271
4,state convention democratic alliance knowledge...,flaws in digital india project,the state convention of democratic alliance f...,5,1,20180311,"[state, convention, democratic, alliance, know...",172


In [28]:
news_df['year'] = news_df['num_date'].map(yr)
news_df['month'] = news_df['num_date'].map(mnth)
news_df['quarter'] = news_df['num_date'].map(qtr)
news_df['day'] = news_df['num_date'].map(day)
news_df['year_quarter'] = news_df['year'] + news_df_topics['quarter']/10
news_df_decriptive = news_df[['year_quarter','num_date','count_tokens','aadhaar_count' ]].copy(deep=True)
news_df_decriptive.sort_values(['year_quarter','count_tokens', 'aadhaar_count'], ascending=[1, 1, 1], inplace = True)
news_df_decriptive.head()


,year_quarter,num_date,count_tokens,aadhaar_count
2612,2006.1,20060131,343,1
2611,2006.1,20060204,349,1
2613,2006.1,20060121,349,1
2610,2006.2,20060415,29,1
2608,2007.1,20070204,276,1


In [29]:
news_df_decriptive['aadhaar_per_100_tokens'] = (100*news_df_decriptive['aadhaar_count' ]/ news_df_decriptive['count_tokens' ]).round(3)

news_df_decriptive['mean_token_count'] = news_df_decriptive.groupby(['year_quarter'])['count_tokens'].transform('mean')

news_df_decriptive['mean_aadhaar_count'] = news_df_decriptive.groupby(['year_quarter'])['aadhaar_count'].transform('mean')

news_df_decriptive['mean_aadhaar_per_100_tokens'] = news_df_decriptive.groupby(['year_quarter'])['aadhaar_per_100_tokens'].transform('mean')

news_df_decriptive['article_occurences'] = news_df_decriptive.groupby(['year_quarter'])['aadhaar_count'].transform('count')

news_df_decriptive['aadhaar_occurences'] = news_df_decriptive.groupby(['year_quarter'])['aadhaar_count'].transform('sum')


news_df_decriptive.drop(['num_date', 'count_tokens', 'aadhaar_count', 'aadhaar_per_100_tokens'], axis = 1, inplace = True)

print(len(news_df_decriptive))
news_df_decriptive.drop_duplicates(inplace=True)
print(len(news_df_decriptive))


2614
41


In [30]:

merge_year_quarter = pd.read_excel('merge_year_quarter.xlsx')

#df_1 = list(plotly_mean_sentiment['topic'].unique())
#df_2 =list(merge_year_quarter['year_quarter'].unique())


#dfn = pd.DataFrame([(x, y) for x in df_1 for y in df_2])
#dfn.rename(columns={1:'year_quarter',0 :'topic' }, inplace=True)
news_df_decriptive= pd.merge(news_df_decriptive,merge_year_quarter,how='right',on=['year_quarter'])
print(len(news_df_decriptive))
news_df_decriptive.sort_values(['year_quarter'], ascending=[1], inplace = True)
news_df_decriptive.fillna(0, inplace = True)
news_df_decriptive.mean_aadhaar_per_100_tokens.round(3)
news_df_decriptive.mean_token_count.round(1)
news_df_decriptive.mean_aadhaar_count.round(1)
news_df_decriptive.head()

49


,year_quarter,mean_token_count,mean_aadhaar_count,mean_aadhaar_per_100_tokens,article_occurences,aadhaar_occurences
0,2006.1,347.0,1.0,0.288667,3.0,3.0
1,2006.2,29.0,1.0,3.448000,1.0,1.0
41,2006.3,0.0,0.0,0.000000,0.0,0.0
42,2006.4,0.0,0.0,0.000000,0.0,0.0
2,2007.1,304.5,1.0,0.331000,2.0,2.0


In [31]:
writer = pd.ExcelWriter('news_df_decriptive_002.xlsx')
news_df_decriptive.to_excel(writer, sheet_name='news_df_decriptive')
writer.save()